In [1]:
import pandas as pd
import seaborn as sns
from pathlib import Path as path
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.impute import KNNImputer

import warnings
warnings.filterwarnings('ignore')

Training Data

In [2]:
Xtrain = pd.read_csv(path('data/Xtrain.csv'))
Ytrain = pd.read_csv(path('data/Ytrain.csv'))
Ytrain = Ytrain.loc[:, ~Ytrain.columns.str.contains('^Unnamed')]
TrainData = Xtrain
TrainData = TrainData.merge(Ytrain, left_index= True, right_index= True)

Data Cleaning

In [3]:
def cleaning(dirty_df):
    # Removing rows with 0 hour
    dirty_df = dirty_df[dirty_df['hour'].isna() == False]
    
    # Add month, day, weekday, and intHour
    dirty_df['date'] = pd.to_datetime(dirty_df['date'], format='%Y/%m/%d')
    weekday = dirty_df['date'].dt.weekday
    dirty_df.insert(1,'weekday',weekday)
    month = dirty_df['date'].dt.month
    dirty_df.insert(2,'month',month)
    hour = pd.to_datetime(dirty_df['hour'], format='%H:%M:%S').dt.hour
    dirty_df.insert(1,'intHour',hour)
    
    # Add lines
    line = []
    for i in dirty_df["train"]:
        if i <= 11:
            line.append(0)
        elif i <= 21:
            line.append(1)
        elif i <= 27:
            line.append(2)
        elif i == 28:
            line.append(3)
        elif i <= 33:
            line.append(4)
        elif i <= 44:
            line.append(5)
        elif i <= 50:
            line.append(6)
        else:
            line.append(7)

    dirty_df.insert(1,'line',line)
    
    # P*q0
    dirty_df.loc[(dirty_df['p1q0'].isna()) & (dirty_df['p2q0'].isna()) & (dirty_df['p3q0'].isna()), ['p1q0','p2q0','p3q0']] = 0
    dirty_df.loc[(dirty_df['p1q0'].isna() == False) & (dirty_df['p2q0'].isna()) & (dirty_df['p3q0'].isna()), ['p2q0','p3q0']] = 0
    for i, row in dirty_df.iterrows():
        if math.isnan(row['p3q0']):
            if row['p1q0'] > row['p2q0']:
                diff = row['p1q0'] - (row['p1q0'] - row['p2q0'])
            else:
                diff = row['p2q0'] - (row['p2q0'] - row['p1q0'])
            dirty_df.at[i,'p3q0'] = diff
    # p0Q*
    dirty_df.loc[(dirty_df['p0q1'].isna()) & (dirty_df['p0q2'].isna()) & (dirty_df['p0q3'].isna()), ['p0q1','p0q2','p0q3']] = 0
    dirty_df.loc[(dirty_df['p0q1'].isna() == False) & (dirty_df['p0q2'].isna()) & (dirty_df['p0q3'].isna()), ['p0q2','p0q3']] = 0
    for i, row in dirty_df.iterrows():
        if math.isnan(row['p0q3']):
            if row['p0q1'] > row['p0q2']:
                diff = row['p0q1'] - (row['p0q1'] - row['p0q2'])
            else:
                diff = row['p0q2'] - (row['p0q2'] - row['p0q1'])
            dirty_df.at[i,'p0q3'] = diff
            
    # Drop values
    dirty_df = dirty_df.drop('date', axis=1)
    dirty_df = dirty_df.drop('hour', axis=1)
    dirty_df = dirty_df.drop('way', axis=1)
    dirty_df = dirty_df.drop('composition', axis=1)
    dirty_df = dirty_df[dirty_df['station'] != 'AR']
    dirty_df = dirty_df[dirty_df['station'] != 'AZ']

    # Sort by station
    dirty_df = dirty_df.sort_values('station')

    return dirty_df

In [4]:
TrainData = cleaning(TrainData)
TrainData

,line,intHour,weekday,month,train,station,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,p0q0
23231,6,8,4,3,48,AA,0.049000,0.05800,0.1190,0.036,0.046,0.087,0.112
21880,6,6,0,5,46,AA,0.045000,0.00000,0.0000,0.017,0.021,0.057,0.079
21881,6,6,3,3,46,AA,0.047000,0.00000,0.0000,0.025,0.030,0.069,0.088
21882,6,6,2,1,46,AA,0.047000,0.00000,0.0000,0.025,0.030,0.045,0.065
22493,6,7,0,1,47,AA,0.044000,0.08235,0.0440,0.048,0.057,0.105,0.139
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23848,6,8,1,1,48,BJ,0.050000,0.10900,0.1660,0.129,0.049,0.099,0.146
23849,6,8,2,1,48,BJ,0.045333,0.08400,0.1755,0.124,0.042,0.094,0.143
23850,6,8,3,1,48,BJ,0.048000,0.16800,0.1450,0.114,0.061,0.087,0.132
23852,6,8,0,1,48,BJ,0.062000,0.08000,0.1520,0.134,0.052,0.102,0.153


Label Encoding Station

In [5]:
def label_enc(param):
    LabelE = LabelEncoder()
    LabelE.fit(param)
    print(param.name,LabelE.classes_)
    return LabelE.transform(param)

In [6]:
TrainData["station"] = label_enc(TrainData["station"])
TrainData

station ['AA' 'AB' 'AC' 'AD' 'AE' 'AF' 'AG' 'AH' 'AI' 'AJ' 'AK' 'AL' 'AM' 'AN'
 'AO' 'AP' 'AQ' 'AS' 'AT' 'AU' 'AW' 'AY' 'BA' 'BB' 'BC' 'BD' 'BE' 'BF'
 'BG' 'BH' 'BI' 'BJ']


,line,intHour,weekday,month,train,station,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,p0q0
23231,6,8,4,3,48,0,0.049000,0.05800,0.1190,0.036,0.046,0.087,0.112
21880,6,6,0,5,46,0,0.045000,0.00000,0.0000,0.017,0.021,0.057,0.079
21881,6,6,3,3,46,0,0.047000,0.00000,0.0000,0.025,0.030,0.069,0.088
21882,6,6,2,1,46,0,0.047000,0.00000,0.0000,0.025,0.030,0.045,0.065
22493,6,7,0,1,47,0,0.044000,0.08235,0.0440,0.048,0.057,0.105,0.139
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23848,6,8,1,1,48,31,0.050000,0.10900,0.1660,0.129,0.049,0.099,0.146
23849,6,8,2,1,48,31,0.045333,0.08400,0.1755,0.124,0.042,0.094,0.143
23850,6,8,3,1,48,31,0.048000,0.16800,0.1450,0.114,0.061,0.087,0.132
23852,6,8,0,1,48,31,0.062000,0.08000,0.1520,0.134,0.052,0.102,0.153


Model P*Q* with Stations

In [7]:
def build_model(drop_columns):
    model = TrainData.copy()
    model = model.drop(columns=drop_columns, axis= 1)
    return model

In [8]:
def train_lr(model, export):
    Xtrain = model.drop(columns=['p0q0'], axis=1)
    Ytrain = model['p0q0']

    x_train, x_test, y_train, y_test = train_test_split(Xtrain, Ytrain, test_size = 0.2, random_state = 42)
    LR = LinearRegression()
    LR.fit(x_train,y_train)
    y_prediction =  LR.predict(x_test)
    
    if (export):
        exportToCSV(y_prediction)
    
    score = r2_score(y_test,y_prediction)
    mse = mean_squared_error(y_test, y_prediction)

    print(f'r2 socre is:            {round(score,2)}    ->  %{round(score*100,2)}')
    print(f'Mean Squard Error is:     {round(mse,2)}    ->  %{round(mse*100,2)}')

In [9]:
def train_dnn(model, epochs, export):
    Xtrain = model.drop(columns=['p0q0'], axis=1)
    Ytrain = model['p0q0']
    x_train, x_test, y_train, y_test = train_test_split(Xtrain, Ytrain, test_size = 0.2, random_state = 42)
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    model=keras.models.Sequential([
        keras.layers.Dense(512, input_dim = x_train.shape[1], activation='relu'),  
        keras.layers.Dense(512, input_dim = x_train.shape[1], activation='relu'),  
        keras.layers.Dense(units=256,activation='relu'),  
        keras.layers.Dense(units=256,activation='relu'),    
        keras.layers.Dense(units=128,activation='relu'),
        keras.layers.Dense(units=1, activation="linear"),
    ],name="Initial_model",)
    
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    checkpoint_name = 'Weights\Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
    callbacks_list = [checkpoint]

    history = model.fit(x_train, y_train,
                    epochs=epochs, batch_size=1024,
                    validation_data=(x_test, y_test), 
                    callbacks=callbacks_list, 
                    verbose=1)
    
    y_prediction = model.predict(x_test)
    
    if (export):
        exportToCSV(y_prediction)
    
    score = r2_score(y_test,y_prediction)
    mse = mean_squared_error(y_test, y_prediction)

    print(f'r2 socre is:            {round(score,2)}    ->  %{round(score*100,2)}')
    print(f'Mean Squard Error is:     {round(mse,2)}    ->  %{round(mse*100,2)}')

In [10]:
def exportToCSV(data):
    data = pd.DataFrame(data).copy()
    data.rename(columns = {0:'p0q0'}, inplace = True)
    data.index = np.arange(1, len(data) + 1)
    data.to_csv("y_prediction.csv")

# LR

In [11]:
droppedColumns = []
model = build_model(droppedColumns)
train_lr(model, False)

r2 socre is:            0.92    ->  %91.72
Mean Squard Error is:     0.0    ->  %0.19


# DNN

In [12]:
droppedColumns = []
model = build_model(droppedColumns)
train_dnn(model, 250, True)

Epoch 1/250
20/22 [==========================>...] - ETA: 0s - loss: 0.6028
Epoch 1: val_loss improved from inf to 0.02072, saving model to Weights\Weights-001--0.02072.hdf5
22/22 [==============================] - 2s 50ms/step - loss: 0.5685 - val_loss: 0.0207
Epoch 2/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0211
Epoch 2: val_loss improved from 0.02072 to 0.01686, saving model to Weights\Weights-002--0.01686.hdf5
22/22 [==============================] - 1s 35ms/step - loss: 0.0211 - val_loss: 0.0169
Epoch 3/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0147
Epoch 3: val_loss improved from 0.01686 to 0.01250, saving model to Weights\Weights-003--0.01250.hdf5
22/22 [==============================] - 1s 36ms/step - loss: 0.0147 - val_loss: 0.0125
Epoch 4/250
22/22 [==============================] - ETA: 0s - loss: 0.0114
Epoch 4: val_loss improved from 0.01250 to 0.00936, saving model to Weights\Weights-004--0.00936.hdf5
22/22 [=====================

21/22 [===========================>..] - ETA: 0s - loss: 0.0012
Epoch 35: val_loss did not improve from 0.00110
22/22 [==============================] - 1s 32ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 36/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0013
Epoch 36: val_loss did not improve from 0.00110
22/22 [==============================] - 1s 30ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 37/250
20/22 [==========================>...] - ETA: 0s - loss: 0.0021
Epoch 37: val_loss did not improve from 0.00110
22/22 [==============================] - 1s 31ms/step - loss: 0.0021 - val_loss: 0.0012
Epoch 38/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0015
Epoch 38: val_loss improved from 0.00110 to 0.00108, saving model to Weights\Weights-038--0.00108.hdf5
22/22 [==============================] - 1s 34ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 39/250
20/22 [==========================>...] - ETA: 0s - loss: 0.0016
Epoch 39: val_loss did not im

Epoch 72/250
20/22 [==========================>...] - ETA: 0s - loss: 0.0011
Epoch 72: val_loss improved from 0.00092 to 0.00090, saving model to Weights\Weights-072--0.00090.hdf5
22/22 [==============================] - 1s 34ms/step - loss: 0.0011 - val_loss: 9.0038e-04
Epoch 73/250
21/22 [===========================>..] - ETA: 0s - loss: 9.2396e-04
Epoch 73: val_loss did not improve from 0.00090
22/22 [==============================] - 1s 32ms/step - loss: 9.2206e-04 - val_loss: 0.0011
Epoch 74/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0011
Epoch 74: val_loss did not improve from 0.00090
22/22 [==============================] - 1s 31ms/step - loss: 0.0011 - val_loss: 0.0015
Epoch 75/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0015
Epoch 75: val_loss did not improve from 0.00090
22/22 [==============================] - 1s 32ms/step - loss: 0.0015 - val_loss: 9.0447e-04
Epoch 76/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0010


Epoch 108/250
22/22 [==============================] - ETA: 0s - loss: 8.9952e-04
Epoch 108: val_loss did not improve from 0.00081
22/22 [==============================] - 1s 31ms/step - loss: 8.9952e-04 - val_loss: 0.0019
Epoch 109/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0013
Epoch 109: val_loss did not improve from 0.00081
22/22 [==============================] - 1s 31ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 110/250
21/22 [===========================>..] - ETA: 0s - loss: 0.0011
Epoch 110: val_loss did not improve from 0.00081
22/22 [==============================] - 1s 34ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 111/250
22/22 [==============================] - ETA: 0s - loss: 9.1098e-04
Epoch 111: val_loss did not improve from 0.00081
22/22 [==============================] - 1s 37ms/step - loss: 9.1098e-04 - val_loss: 9.5503e-04
Epoch 112/250
21/22 [===========================>..] - ETA: 0s - loss: 8.6978e-04
Epoch 112: val_loss did not improve fr

Epoch 144/250
20/22 [==========================>...] - ETA: 0s - loss: 7.9257e-04
Epoch 144: val_loss improved from 0.00076 to 0.00074, saving model to Weights\Weights-144--0.00074.hdf5
22/22 [==============================] - 1s 37ms/step - loss: 7.9325e-04 - val_loss: 7.4374e-04
Epoch 145/250
21/22 [===========================>..] - ETA: 0s - loss: 8.8856e-04
Epoch 145: val_loss did not improve from 0.00074
22/22 [==============================] - 1s 34ms/step - loss: 8.8790e-04 - val_loss: 0.0010
Epoch 146/250
20/22 [==========================>...] - ETA: 0s - loss: 8.5643e-04
Epoch 146: val_loss improved from 0.00074 to 0.00074, saving model to Weights\Weights-146--0.00074.hdf5
22/22 [==============================] - 1s 35ms/step - loss: 8.4981e-04 - val_loss: 7.3665e-04
Epoch 147/250
20/22 [==========================>...] - ETA: 0s - loss: 9.2575e-04
Epoch 147: val_loss did not improve from 0.00074
22/22 [==============================] - 1s 32ms/step - loss: 9.2253e-04 - val_los

Epoch 179/250
21/22 [===========================>..] - ETA: 0s - loss: 6.9096e-04
Epoch 179: val_loss did not improve from 0.00068
22/22 [==============================] - 1s 33ms/step - loss: 6.9097e-04 - val_loss: 7.3789e-04
Epoch 180/250
22/22 [==============================] - ETA: 0s - loss: 7.8495e-04
Epoch 180: val_loss did not improve from 0.00068
22/22 [==============================] - 1s 30ms/step - loss: 7.8495e-04 - val_loss: 7.2039e-04
Epoch 181/250
20/22 [==========================>...] - ETA: 0s - loss: 8.2340e-04
Epoch 181: val_loss did not improve from 0.00068
22/22 [==============================] - 1s 30ms/step - loss: 8.1461e-04 - val_loss: 7.8155e-04
Epoch 182/250
21/22 [===========================>..] - ETA: 0s - loss: 7.0430e-04
Epoch 182: val_loss did not improve from 0.00068
22/22 [==============================] - 1s 29ms/step - loss: 7.0391e-04 - val_loss: 8.0118e-04
Epoch 183/250
20/22 [==========================>...] - ETA: 0s - loss: 6.7945e-04
Epoch 183:

Epoch 214/250
21/22 [===========================>..] - ETA: 0s - loss: 7.4983e-04
Epoch 214: val_loss did not improve from 0.00062
22/22 [==============================] - 1s 32ms/step - loss: 7.5154e-04 - val_loss: 6.3941e-04
Epoch 215/250
20/22 [==========================>...] - ETA: 0s - loss: 6.0579e-04
Epoch 215: val_loss did not improve from 0.00062
22/22 [==============================] - 1s 30ms/step - loss: 6.0642e-04 - val_loss: 6.9756e-04
Epoch 216/250
19/22 [========================>.....] - ETA: 0s - loss: 6.5911e-04
Epoch 216: val_loss did not improve from 0.00062
22/22 [==============================] - 1s 30ms/step - loss: 6.5262e-04 - val_loss: 6.2990e-04
Epoch 217/250
20/22 [==========================>...] - ETA: 0s - loss: 6.9692e-04
Epoch 217: val_loss did not improve from 0.00062
22/22 [==============================] - 1s 33ms/step - loss: 6.9044e-04 - val_loss: 8.6621e-04
Epoch 218/250
22/22 [==============================] - ETA: 0s - loss: 9.6414e-04
Epoch 218:

Epoch 249/250
20/22 [==========================>...] - ETA: 0s - loss: 5.6600e-04
Epoch 249: val_loss did not improve from 0.00056
22/22 [==============================] - 1s 30ms/step - loss: 5.6066e-04 - val_loss: 5.8059e-04
Epoch 250/250
20/22 [==========================>...] - ETA: 0s - loss: 5.6806e-04
Epoch 250: val_loss did not improve from 0.00056
22/22 [==============================] - 1s 30ms/step - loss: 5.6870e-04 - val_loss: 6.5924e-04
r2 socre is:            0.97    ->  %97.09
Mean Squard Error is:     0.0    ->  %0.07
